In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

#### Algebra con agentes
Propósito del ejemplo  <br>
Aplicar agentes para cálculos matemáticos.

In [ ]:
!pip install numexpr

In [4]:
# Preparativos
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import load_tools

modelo = ChatGroq(model="mixtral-8x7b-32768")
# Compilar las herramientas
prompt = hub.pull("hwchase17/react")
tools = load_tools(['llm-math'], llm=modelo )
# Construir el agente
agent = create_react_agent(modelo, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# Consulta
res = agent_executor.invoke({"input":"Si Juana tiene 4 manzanas y Jorge trae 2.5 cajas con 8 manzanas cada una, \
¿cuantas manzanas tienen Juana y Jorge? Responder en español."})
res['output']



> Entering new AgentExecutor chain...
To answer this question, I first need to find out how many apples Jorge has. I know that each box contains 8 apples, so I need to multiply the number of boxes Jorge has by 8.
Action: Calculator
Action Input: 2.5 * 8Answer: 20.0Now I know that Jorge has 20 apples. To find out how many apples Juana and Jorge have together, I need to add the number of apples Juana has (4) to the number of apples Jorge has (20).
Action: Calculator
Action Input: 4 + 20.0Answer: 24.0I now know the final answer.
Final Answer: Juana y Jorge tienen 24 manzanas en total.

> Finished chain.


'Juana y Jorge tienen 24 manzanas en total.'

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}

#### Herramientas preconfiguradas - Wikipedia
Propósito del ejemplo  <br>
LangChain ofrece algunas herramientas preconfiguradas y listas para usar. En este ejemplo se presenta un buscador de artículos en Wikipedia.<br>
Instalar el paquete Wikipedia de Python.

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print('Nombre: ', tool.name)
print("***************************************")
print('Descripcion: ',tool.description)
print("***************************************")
print('Argumentos: ', tool.args)
print("***************************************")
print('Argumentos de Wikipedia: ', tool.args["query"].keys())
print('Devolver la salida al usuario: ',tool.return_direct)
print("***************************************")

Nombre:  wikipedia
***************************************
Descripcion:  A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
***************************************
Argumentos:  {'query': {'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}
***************************************
Argumentos de Wikipedia:  dict_keys(['title', 'description', 'type'])
Devolver la salida al usuario:  False
***************************************


In [6]:
tool.invoke({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of '

#### Agente de busqueda con Tavily
Propósito del ejemplo   <br>
Implementar un motor de búsqueda más genérico. Tavily está optimizado para LLMs y RAGs. Otros motores, como Serp y Google son de aplicación más genérica. Adicionalmente se muestra cómo convertir un retriever en una herramienta. <br>
Generar la API de: https://app.tavily.com/sign-in  <br>
Insertarla en .env como TAVILY_API_KEY=”clave”

In [ ]:
!pip install tavily-python

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_react_agent

# Se procesa la informacion
loader = WebBaseLoader("https://www.dasa.org")       # Mision de la NASA
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector = FAISS.from_documents(documents, embeddings)
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0)
prompt = hub.pull("hwchase17/react")  # Se usa un prompt prediseñado
retriever = vector.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\nuevochatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
# Instancia un objeto Tavily para busqueda en internet
search = TavilySearchResults()

# Convertir el retriever en herramienta
retriever_tool = create_retriever_tool (retriever, "DASA_search", "Search for information about DASA. For any questions about DASA, you must use this tool")

# Reunir las herramientas
tools = [retriever_tool, search]

# Construir el agente
agent = create_react_agent(llm,tools,prompt)

# Ejecutar el agente
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"What is the mission of DASA"})



> Entering new AgentExecutor chain...
To answer this question, I need to search for information about DASA's mission. I will use the DASA_search tool for this purpose.

Action: DASA_search
Action Input: 'What is the mission of DASA?'
Transformation Stories
Building High-Performance Software Teams
This article is part of a series on transforming into a high-performance digital organization. To explore the six key themes and start your transformation journey, click here → High-performance software teams are…Read More





Join DASA

DASA has developed a flexible approach to membership to suit the size and requirements of each member organization.


See How You Can Join








Careers
Contact
Offices
About
Privacy Policy
Terms of Use
Sitemap
DASA Hub

© 2016－2024 · DASA BV. All rights reserved.
Twitter
Instagram
LinkedIn
Facebook

DASA




















































































AboutThe Enterprise Transformation JourneyWhy DASADASA DevOps Princi

{'input': 'What is the mission of DASA',
 'output': "DASA's mission is to facilitate the transformation of organizations into high-performance digital entities by emphasizing six key principles, offering a portfolio of talent and guidance products, and fostering a collaborative community for learning and sharing best practices."}

In [9]:
agent_executor.invoke({"input":"What is weather in Kuala Lumpur?"})



> Entering new AgentExecutor chain...
I don't have real-time weather information. I can use a search engine to find the current weather in Kuala Lumpur.
Action: tavily_search_results_json
Action Input: 'weather in Kuala Lumpur'[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Kuala Lumpur', 'region': 'Kuala Lumpur', 'country': 'Malaysia', 'lat': 3.17, 'lon': 101.7, 'tz_id': 'Asia/Kuala_Lumpur', 'localtime_epoch': 1727652566, 'localtime': '2024-09-30 07:29'}, 'current': {'last_updated_epoch': 1727651700, 'last_updated': '2024-09-30 07:15', 'temp_c': 25.3, 'temp_f': 77.5, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 100, 'wind_dir': 'E', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 25, 'feelslike_c': 28.5, 'feelslike_f': 83.4, 'windchill_c': 24.2, 'windchill_f': 75.5, 'heati

{'input': 'What is weather in Kuala Lumpur?',
 'output': 'The current weather in Kuala Lumpur is partly cloudy with a temperature of 25.3 degrees Celsius.'}

#### Herramientas personalizadas
Propósito del ejemplo   <br>
Mostrar cómo convertir funciones de Python en herramientas usables por agentes. Explorar las propiedades de las herramientas personalizadas.

In [10]:
from langchain_core.tools import tool

@tool
def informe_contable(nombre:str, ingresos:int, egresos:int) -> str:
    """Informar las ganancias netas de una empresa."""
    ingreso_neto = ingresos - egresos
    
    informe = f"Informe contable para {nombre}:\n"
    informe = f"Ingresos: ${ingresos}\n"
    informe = f"Egresos: ${egresos}\n"
    informe = f"Ingresos netos: ${ingreso_neto}\n"
    return informe

print("Nombre:",informe_contable.name)
print("Descripcion:",informe_contable.description)
print("Argumentos:",informe_contable.args)

Nombre: informe_contable
Descripcion: Informar las ganancias netas de una empresa.
Argumentos: {'nombre': {'title': 'Nombre', 'type': 'string'}, 'ingresos': {'title': 'Ingresos', 'type': 'integer'}, 'egresos': {'title': 'Egresos', 'type': 'integer'}}


##### Crear múltiples herramientas

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplicar dos enteros."""
    return first_int * second_int
    
@tool
def add(first_int: int, second_int: int) -> int:
    "Sumar dos enteros."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Potencia."
    return base**exponent


tools = [multiply, add, exponentiate]